## Chapter 3: Convolutional Neural Networks

You'll need to either copy the image training files you downloaded in Chapter 2 to this directory, or alter the paths appropriately.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image

## CNNNet (or AlexNet by another name…)

In [11]:
class CNNNet(nn.Module):

    def __init__(self, num_classes=2):
        super(CNNNet, self).__init__()

        # Define the convolutional layers
        self.features = nn.Sequential(
            # in_channels = number of input channels as input image is RGB it is 3
            # out_channels = number of output channels equals to number of filters in conv layer
            # kernel_size = height and width of the filter (11*11) can be scaler or tuple like (3,5)
            # stride = indicates how many steps across the input we move when we adjust the filter to a new position
            # padding will add a zero padding to the input array
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

        # Define the adaptive average pooling layer
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        # Define the fully connected layers
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        # Forward pass through the network
        x = self.features(x)  # Apply convolutional layers
        x = self.avgpool(x)   # Apply adaptive average pooling
        x = torch.flatten(x, 1)  # Flatten the tensor
        x = self.classifier(x)  # Apply fully connected layers
        return x

In [12]:
cnnnet = CNNNet()

In [13]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1],
                               targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [14]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

img_transforms = transforms.Compose([
    transforms.Resize((64,64)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
    ])
train_data_path = "../Chapter 2/train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)
val_data_path = "../Chapter 2/val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)
batch_size=64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,shuffle=True)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)

if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

In [15]:
cnnnet.to(device)
optimizer = optim.Adam(cnnnet.parameters(), lr=0.001)

In [16]:
train(cnnnet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=10, device=device)

Epoch: 1, Training Loss: 1.74, Validation Loss: 0.67, accuracy = 0.84
Epoch: 2, Training Loss: 0.69, Validation Loss: 0.66, accuracy = 0.84
Epoch: 3, Training Loss: 0.67, Validation Loss: 0.98, accuracy = 0.28
Epoch: 4, Training Loss: 0.61, Validation Loss: 0.41, accuracy = 0.79
Epoch: 5, Training Loss: 0.51, Validation Loss: 0.68, accuracy = 0.56
Epoch: 6, Training Loss: 0.50, Validation Loss: 0.32, accuracy = 0.85
Epoch: 7, Training Loss: 0.50, Validation Loss: 0.73, accuracy = 0.58
Epoch: 8, Training Loss: 0.53, Validation Loss: 0.57, accuracy = 0.65
Epoch: 9, Training Loss: 0.47, Validation Loss: 0.38, accuracy = 0.81
Epoch: 10, Training Loss: 0.45, Validation Loss: 0.49, accuracy = 0.72


## Downloading a pretrained network
There are two ways of downloading pre-trained image models with PyTorch. Firstly, you can use the `torchvision.models` library, or you can use PyTorch Hub. The latter is preferred as of 2019, as this is a one-stop shop for all models and the new standard for distributing models with PyTorch.

In [ ]:
import torchvision.models as models
alexnet = models.alexnet(num_classes=1000, pretrained=True)

D:\Software\Python Enviorments\pytorch_venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Software\Python Enviorments\pytorch_venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\rsani/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
10.1%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--Ser